## DC-GAN 

- Paper - https://arxiv.org/abs/1511.06434
- Abstract - In recent years, supervised learning with convolutional networks (CNNs) has seen huge adoption in computer vision applications. Comparatively, unsupervised learning with CNNs has received less attention. In this work we hope to help bridge the gap between the success of CNNs for supervised learning and unsupervised learning. We introduce a class of CNNs called deep convolutional generative adversarial networks (DCGANs), that have certain architectural constraints, and demonstrate that they are a strong candidate for unsupervised learning. Training on various image datasets, we show convincing evidence that our deep convolutional adversarial pair learns a hierarchy of representations from object parts to scenes in both the generator and discriminator. Additionally, we use the learned features for novel tasks - demonstrating their applicability as general image representations.

Importing Neccesary libraries

In [1]:
import argparse
import os
import numpy as np 
import math 
import torchvision.transforms as transforms 
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn 
import torch.nn.functional as F 
import torch 

os.makedirs("images",exist_ok=True)

In [60]:
# constants
n_epochs = 200
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 100
img_size = 32
channels = 1
sample_interval = 400
img_shape = (channels, img_size, img_size)
if torch.cuda.is_available():
  cuda = True
else:
  cuda = False 

What is nn.Module ?

- https://github.com/torch/nn/blob/master/doc/module.md

- https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/module.py

What is super(Generator,self).__init__() ?

- https://stackoverflow.com/a/33469090

- https://stackoverflow.com/questions/576169/understanding-python-super-with-init-methods

nn.linear 

- https://pytorch.org/docs/stable/generated/torch.nn.Linear.html




In [61]:

def weigths_init_normal(m):
  classname = m.__class__.__name__
  if classname.find("Conv") != -1:
    #print("reach")
    torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif classname.find("BatchNorm2d") != -1:
    #print("reach2")
    torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
    torch.nn.init.constant_(m.bias.data, 0.0)



class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.init_size = img_size // 4 
    self.l1 = nn.Sequential(nn.Linear(latent_dim, 128*self.init_size**2 ))
    #self.l1.shape
    #print(self.l1.shape[0], 128, self.init_size, self.init_size)
    self.conv_blocks = nn.Sequential(
        nn.BatchNorm2d(128),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(128,128,3, stride=1, padding =1),
        nn.BatchNorm2d(128,0.8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(128,64,3, stride=1, padding=1),
        nn.BatchNorm2d(64,0.8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(64, channels, 3, stride=1, padding=1),
        nn.Tanh(),
    )

    
  def forward(self, z):
    out = self.l1(z)
    # reshaping the img
    out = out.view(out.shape[0], 128, self.init_size, self.init_size)
    print(out.shape)
    img = self.conv_blocks(out)
    return img 

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    def discriminator_block(in_filters, out_filters, bn=True):
      block = [nn.Conv2d(in_filters, out_filters, 3,2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
      if bn:
        block.append(nn.BatchNorm2d(out_filters, 0.8))
      return block 
    self.model = nn.Sequential(
        *discriminator_block(channels, 16, bn=False),
        *discriminator_block(16, 32),
        *discriminator_block(32, 64),
        *discriminator_block(64, 128),
    )
    ds_size = img_size //2 ** 4
    self.adv_layer = nn.Sequential(nn.Linear(128*ds_size**2, 1), nn.Sigmoid())
    
  def forward(self, img):
    # flattening tensors
    out = self.model(img)
    out = out.view(out.shape[0], -1)
    validity = self.adv_layer(out)
    return validity



In [62]:
# loss function
adversarial_loss = torch.nn.BCELoss()

# initalizing generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [63]:


if cuda:
  generator.cuda()
  discriminator.cuda()
  adversarial_loss.cuda()

generator.apply(weigths_init_normal)
discriminator.apply(weigths_init_normal)

# data loader
os.makedirs("data/mnist",exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist",
        train = True,
        download = True,
        transform = transforms.Compose( \
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5],[0.5])]
        ),
    ),
    batch_size = batch_size,
    shuffle = True,
)


#optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1,b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1,b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor




## Training

In [ ]:
for epoch in range(n_epochs):
  for i, (imgs, _) in enumerate(dataloader):
    #ground truths definition
    # valid 
    valid = Variable(Tensor(imgs.size(0),1).fill_(1.0), requires_grad=False )
    # fake 
    fake = Variable(Tensor(imgs.size(0),1).fill_(0.0), requires_grad= False)

    # Configure inuput 
    real_imgs = Variable(imgs.type(Tensor))

    ########## Train generator ################

    # clears the output of all variables from previous iteration
    optimizer_G.zero_grad()

    # sample noise as gen input
    z = Variable(Tensor(np.random.normal(0,1,(imgs.shape[0], latent_dim))))
    print(z.shape)

    gen_imgs = generator(z)
    # checking how much genrators image the discrimainator is able to classify as valid and then taking it as a loss and back propagating
    g_loss = adversarial_loss(discriminator(gen_imgs), valid)
    # backpropagation
    g_loss.backward()

    ######## training discriminator ###############

    optimizer_D.zero_grad()

    real_loss = adversarial_loss(discriminator(real_imgs), valid)

    # tensor.detach() creates a tensor that shares storage with tensor that does not require grad. It detaches the output from the computational graph
    # https://stackoverflow.com/questions/56816241/difference-between-detach-and-with-torch-nograd-in-pytorch#:~:text=detach()%20creates%20a%20tensor,output%20from%20the%20computational%20graph.

    fake_loss = adversarial_loss(discriminator(gen_imgs.detach()),fake)
    d_loss = (real_loss + fake_loss)/2 
    # backpropagation
    d_loss.backward()

    optimizer_D.step()

    print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" %
          (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item()))
    
    batches_done = epoch * len(dataloader) + i 
    if batches_done % sample_interval == 0:
      save_image(gen_imgs.data[25], "images/%d.png"%batches_done, nrow=5, normalize=True)





### End of notebook

###File based code

In [50]:
%%writefile dcgan.py


import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)

parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval between image sampling")
opt = parser.parse_args()
print(opt)

cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

Writing decgan.py


In [51]:
!python decgan.py

Namespace(b1=0.5, b2=0.999, batch_size=64, channels=1, img_size=32, latent_dim=100, lr=0.0002, n_cpu=8, n_epochs=200, sample_interval=400)
9920512it [00:01, 9314936.23it/s]                 
Extracting ../../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw
32768it [00:00, 135006.94it/s]
Extracting ../../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw
1654784it [00:00, 2586147.76it/s]               
Extracting ../../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw
8192it [00:00, 47849.46it/s]
Extracting ../../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw
Processing...
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array

### End of Code